[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/jotabruno/CLIMATE_CHANGE_CMIP6/blob/main/1GCM_CMIP6_TEST.ipynb
)


In [1]:
# Librerias requeridas para instalar, usamos pip
!pip install intake intake-esm
!pip install xmip
!pip install matplotlib
!pip install aiohttp requests
!pip install Cartopy
!pip install icclim
#!pip -q install --upgrade --force-reinstall "fsspec[http]==2025.3.0" "gcsfs==2025.3.0"
print("ok")

  Using cached zarr-3.1.5-py3-none-any.whl.metadata (10 kB)
Using cached zarr-3.1.5-py3-none-any.whl (284 kB)
  Attempting uninstall: zarr
    Found existing installation: zarr 2.18.7
    Uninstalling zarr-2.18.7:
      Successfully uninstalled zarr-2.18.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rechunker 0.5.4 requires zarr<3,>=2.11, but you have zarr 3.1.5 which is incompatible.
  Using cached zarr-2.18.7-py3-none-any.whl.metadata (5.8 kB)
Using cached zarr-2.18.7-py3-none-any.whl (211 kB)
  Attempting uninstall: zarr
    Found existing installation: zarr 3.1.5
    Uninstalling zarr-3.1.5:
      Successfully uninstalled zarr-3.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kerchunk 0.2.9 requires zarr>=3.0.1, but you have

In [2]:
import intake
import matplotlib.pyplot as plt
import xmip
from xmip.preprocessing import combined_preprocessing
import gcsfs
import numpy as np
import random
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io import shapereader
import icclim
import xarray as xr
import xclim.core.calendar as xcal
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
from xmip.preprocessing import combined_preprocessing
import fsspec

fsspec.config.conf["gcs"] = {"token": "anon"}

ModuleNotFoundError: No module named 'cartopy'

In [ ]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
cat = intake.open_esm_datastore(cat_url)

In [ ]:
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
...,...,...,...,...,...,...,...,...,...,...,...
514813,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,<NA>,20211207
514814,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tauu,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,<NA>,20211207
514815,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hur,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,<NA>,20211207
514816,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hus,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,<NA>,20211207


## ¿Qué es CMIP6?

CMIP6 (Coupled Model Intercomparison Project – Phase 6) es la sexta fase del esfuerzo internacional coordinado por el WCRP, cuyo objetivo es comparar y evaluar modelos climáticos globales acoplados (atmósfera–océano–tierra–criósfera).
CMIP6 proporcionó simulaciones históricas, idealizadas y de escenarios futuros (SSP), ampliamente usadas en el IPCC AR6, estudios de cambio climático, impactos y adaptación.

Los datos se organizaron en MIPs (Model Intercomparison Projects), cada uno con objetivos científicos específicos.

# `Descripción de las columnas del catálogo CMIP6 (cat.df)`
## `activity_id`

**Indicó el MIP al que pertenecía la simulación.**
Ejemplos frecuentes en CMIP6:

CMIP: simulaciones históricas y de control (núcleo del proyecto).

ScenarioMIP: proyecciones futuras basadas en escenarios SSP.

HighResMIP: experimentos de alta resolución espacial.

CFMIP: retroalimentaciones climáticas.

DAMIP: atribución y detección.

DCPP: predicción decadal.

PMIP: paleoclima.

C4MIP: ciclo del carbono.

OMIP: océano.

LUMIP: uso del suelo.

Cada activity_id definió el marco científico del experimento.

In [ ]:
# Lista de activity_id
cat.df['activity_id'].unique()

<ArrowExtensionArray>
[ 'HighResMIP',        'CMIP',       'CFMIP', 'ScenarioMIP',  'AerChemMIP',
       'RFMIP',      'FAFMIP',       'DAMIP',       'LUMIP',      'CDRMIP',
       'GMMIP',       'C4MIP',        'OMIP',        'PMIP',      'LS3MIP',
        'DCPP',       'PAMIP',      'ISMIP6']
Length: 18, dtype: large_string[pyarrow]

## `institution_id`

**Institución o consorcio desarrollador del modelo climático.**

Ejemplos:
- ECMWF, MOHC, NOAA-GFDL, IPSL, NCAR, CNRM-CERFACS, MIROC, MPI-M, CMCC.

Este campo permitió identificar la procedencia institucional de cada simulación y evaluar diferencias asociadas al desarrollo del modelo.


In [ ]:
# Lista de institution_id
cat.df['institution_id'].unique()

<ArrowExtensionArray>
[               'CMCC', 'EC-Earth-Consortium',                'MOHC',
               'ECMWF',           'NOAA-GFDL',                'IPSL',
        'E3SM-Project',        'CNRM-CERFACS',           'NASA-GISS',
                 'BCC',               'MIROC',                 'AWI',
                'NCAR',                 'MRI',                 'SNU',
                 'CAS',               'CCCma',                 'INM',
               'NUIST',   'HAMMOZ-Consortium',                'CAMS',
               'MPI-M',                'DKRZ',                 'DWD',
                  'UA',            'NIMS-KMA',                 'NCC',
            'FIO-QLNM',               'KIOST',        'CSIRO-ARCCSS',
               'CSIRO',           'CCCR-IITM',                 'THU',
             'AS-RCEC',                'NERC',             'RUBISCO']
Length: 36, dtype: large_string[pyarrow]

## `source_id`

**Indicó el nombre del modelo climático.**
Ejemplos:

EC-Earth3, CNRM-CM6-1, IPSL-CM6A-LR, MIROC6, CESM2, NorESM2-MM.

Cada source_id corresponde a una configuración específica del modelo (resolución, física, acoplamientos).

In [ ]:
# Lista de source_id
cat.df['source_id'].unique()

<ArrowExtensionArray>
[      'CMCC-CM2-HR4',      'EC-Earth3P-HR',    'HadGEM3-GC31-MM',
    'HadGEM3-GC31-HM',    'HadGEM3-GC31-LM',         'EC-Earth3P',
       'ECMWF-IFS-HR',       'ECMWF-IFS-LR',    'HadGEM3-GC31-LL',
      'CMCC-CM2-VHR4',           'GFDL-CM4',           'GFDL-AM4',
       'IPSL-CM6A-LR',           'E3SM-1-0',         'CNRM-CM6-1',
          'GFDL-ESM4',         'GFDL-ESM2M',       'GFDL-CM4C192',
        'GFDL-OM4p5B',        'GISS-E2-1-G',        'GISS-E2-1-H',
        'CNRM-ESM2-1',        'BCC-CSM2-MR',           'BCC-ESM1',
             'MIROC6',      'AWI-CM-1-1-MR',       'EC-Earth3-LR',
   'IPSL-CM6A-ATM-HR',              'CESM2',        'CESM2-WACCM',
      'CNRM-CM6-1-HR',         'MRI-ESM2-0',        'SAM0-UNICON',
     'GISS-E2-1-G-CC',        'UKESM1-0-LL',          'EC-Earth3',
      'EC-Earth3-Veg',        'FGOALS-f3-L',            'CanESM5',
      'CanESM5-CanOE',          'INM-CM4-8',          'INM-CM5-0',
              'NESM3',    'MPI-ESM-1-2-H

## `experiment_id`

**Definió el experimento climático ejecutado.**
Ejemplos comunes:

historical: clima pasado forzado por observaciones.

piControl: clima preindustrial sin forzantes externos.

1pctCO2, abrupt-4xCO2: experimentos idealizados.

ssp126, ssp245, ssp370, ssp585: escenarios futuros socioeconómicos.

amip: atmósfera forzada con SST observada.

Este campo indica qué forzamiento climático se estaba analizando.

In [ ]:
# Lista de experiment_id
cat.df['experiment_id'].unique()

<ArrowExtensionArray>
[ 'highresSST-present',           'piControl',        'control-1950',
           'hist-1950',          'historical',                'amip',
        'abrupt-4xCO2',        'abrupt-2xCO2',      'abrupt-0p5xCO2',
             '1pctCO2',
 ...
        'pa-futArcSIC',            'pa-pdSIC',      'historical-ext',
  'pdSST-futArcSICSIT', 'pdSST-futOkhotskSIC',  'pdSST-futBKSeasSIC',
         'pa-piArcSIC',         'pa-piAntSIC',        'pa-futAntSIC',
      'pdSST-pdSICSIT']
Length: 170, dtype: large_string[pyarrow]

## `member_id`

**Representó el miembro del ensamble del modelo, por ejemplo:**

r1i1p1f1

donde:

r: realización (condiciones iniciales),

i: inicialización,

p: física,

f: forzamiento.

Permite el análisis de incertidumbre interna y estructural.

In [ ]:
# Lista de  member_id
cat.df['member_id'].unique()

<ArrowExtensionArray>
[   'r1i1p1f1',    'r1i3p1f1',    'r1i2p1f1',    'r1i1p1f2',    'r2i1p1f1',
    'r3i1p1f1',    'r9i1p1f1',    'r8i1p1f1',   'r30i1p1f1',   'r29i1p1f1',
 ...
  'r4i1p1f242',  'r3i1p1f242', 'r29i1p1f242', 'r10i1p1f242', 'r13i1p1f242',
 'r12i1p1f242', 'r11i1p1f242', 'r1i1000p1f2',    'r1i1p2f2',   'r1i1p11f1']
Length: 657, dtype: large_string[pyarrow]

## `table_id`

**Indica la frecuencia temporal y dominio físico del dato.**
Ejemplos:

Amon: variables atmosféricas mensuales.

day: datos diarios.

3hr, 6hrPlev: datos subdiarios.

Omon: océano mensual.

Lmon: superficie terrestre.

fx: campos fijos (topografía, máscaras).

Definió cómo y cada cuánto se almacenan la variable.

In [ ]:
# Lista de table_id
cat.df['table_id'].unique()

<ArrowExtensionArray>
[       'Amon',     '6hrPlev',         '3hr',         'day',       'EmonZ',
        'E3hr',   '6hrPlevPt',      'AERmon',       'LImon',       'CFmon',
        'Lmon',          'fx',       'SImon',         'Ofx',        'Omon',
       'EdayZ',        'Emon',       'CFday',      'AERday',        'Eday',
         'Oyr',         'Eyr',        'Oday',       'SIday',     'AERmonZ',
      '6hrLev', 'E1hrClimMon',       'CF3hr',       'AERhr',        'Odec',
       'Oclim',         'Efx',       'Aclim',      'SIclim',      'IfxGre',
     'ImonGre',       'Eclim']
Length: 37, dtype: large_string[pyarrow]

## `variable_id`

**Identifica la variable climática almacenada.**
Ejemplos:

tas: temperatura del aire a 2 m.

pr: precipitación.

psl: presión a nivel del mar.

rsds: radiación solar entrante.

hus, hur: humedad específica y relativa.

Variables oceánicas, de hielo marino, aerosoles y biogeoquímica.

Cada variable_id sigue el estándar CF Conventions.

In [ ]:
# Lista de variable_id
cat.df['variable_id'].unique()

<ArrowExtensionArray>
[               'ps',              'rsds',              'rlus',
              'rlds',               'psl',              'hurs',
              'huss',               'hus',              'hfss',
              'rsus',
 ...
        'sidmassdyn',    'siforcecoriolx',    'siforceintstry',
 'ocontemppsmadvect',          'difmxylo',       'msftyrhompa',
       'msftmrhompa',            'phabio',           'co3abio',
          'difmxybo']
Length: 700, dtype: large_string[pyarrow]

## `grid_label`

**Describe el tipo de grilla espacial:**

gn: grilla nativa del modelo.

gr: grilla regular interpolada.

gr1, gr2, gm: distintas configuraciones de regrillado.

sufijo z: datos zonales o especiales.

Este campo fue clave para análisis espaciales y regrillado.

In [ ]:
# Lista de grid_label
cat.df['grid_label'].unique()

<ArrowExtensionArray>
['gn', 'gr', 'gr1', 'grz', 'gr2', 'gr2z', 'gr1z', 'gr3', 'gm', 'gnz']
Length: 10, dtype: large_string[pyarrow]

## `zstore`

**Contiene la ruta al almacenamiento en la nube (Zarr) donde se ubicaron los datos, por ejemplo:**
gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/...

Permite el acceso remoto eficiente vía intake-esm, sin descargar archivos completos.

## `dcpp_init_year`

**Indicó el año de inicialización para experimentos decadal (DCPP).**
Fue NA para la mayoría de experimentos no decadales.

## `version`

**Representa la versión de publicación del dataset (fecha AAAAMMDD).**
Permitió identificar actualizaciones o correcciones de datos.